In [ ]:

# %load_ext tensorboard
# %tensorboard --logdir runs

# Set YOLO settings for TensorBoard
from ultralytics import settings
settings.update({'tensorboard': True})

In [ ]:
from ultralytics import YOLO
import os

CONFIG_YAML_FOLDER = 'dataset/training/config_yamls'
K = 4

base_model = 'yolo11'
sizes = ['n', 's', 'm', 'l'] 
models = {'_det': '', '_seg': '-seg', '_pose': '-pose'}
extensions = ['.pt', '.yaml']

project_name = {'_det': 'pieces', '_seg': 'board', '_pose': 'board'}

batch_sizes = {
    'n': 8,  # Nano
    's': 2,  # Small
    'm': 1,  # Medium
    'l': 1,  # Large
}

# Iterate over all combinations of model sizes, types, and extensions
for e in extensions:
    for m in models:
        for s in sizes:
            model_name = f"{base_model}{s}{models[m]}{e}"
            for k in range(K):
                model = YOLO(model_name)
                config_yaml = os.path.join(CONFIG_YAML_FOLDER, f'fold_{k}{m}.yaml')
                                
                model.train(
                    data = config_yaml,
                    epochs = 100,
                    time = 1, # Set max time
                    patience = 20, # Early stopping if no improvement
                    imgsz=640,
                    batch=batch_sizes[s],
                    device = '0',  # Use GPU
                    save = True,
                    save_period = 1,
                    exist_ok = True,
                    project = f'runs/{project_name[m]}',
                    name=f'{base_model}{s}{models[m]}_k{k}_{e.strip(".")}',
                    plots=True,  # Enable training plots
                    val=True,     # Enable validation for metrics               )
                )